#Transformer-Based Sentiment Analysis

##Goal of the project:

Learn how NLP/AI can be used to solve real-world problems. We have the data for 49 million tweets, pre-dominantly focused on the Pakistani Twitter sphere. We want to use this data to figure a few things out.

We want to figure out
Twitter accounts that are biased in favour/against of a specific party (i.e. tweets in favour of a specific political party)

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv("/content/drive/MyDrive/sample.csv")
df_raw = pd.read_csv("/content/drive/MyDrive/sample.csv")

**EDA (EXPLORATORY DATA ANALYSIS)**

In [4]:
df

,index,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,display_text_range,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,withheld_in_countries,extended_tweet,created_day
0,0,2022-04-26 10:37:27+00:00,1518902068735533056,1518902068735533056,RT @fawadchaudhry: آج الیکشن کمیشن کے دفاتر کے...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26
1,1,2022-04-26 10:37:27+00:00,1518902068743913472,1518902068743913472,RT @ImranRiazKhan: عمران خان کی تقریر کا پاکست...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26
2,2,2022-04-26 10:37:27+00:00,1518902068878192640,1518902068878192640,RT @InsafPK: پاکستان تحریک انصاف پنجاب کا صوبا...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26
3,3,2022-04-26 10:37:27+00:00,1518902068953681922,1518902068953681920,RT @ZBasharit: پاکستان رمضان کے مہینے میں آزاد...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26
4,4,2022-04-26 10:37:27+00:00,1518902069050380290,1518902069050380288,RT @ImranRiazKhan: مبینہ طور پر ٹویٹر کی تاریخ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,2022-04-26 10:37:47+00:00,1518902151338020864,1518902151338020864,RT @SadiaAmber10: #امپورٹڈ_حکومت_نامنظور since...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,{'media': array([{'additional_media_info': Non...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26
996,996,2022-04-26 10:37:47+00:00,1518902151191351297,1518902151191351296,نیوٹرل بھڑوے باجوے نے مدد کی چوروں کی اور اپنی...,"<a href=""http://twitter.com/download/iphone"" r...",True,NaN,NaN,NaN,...,NaN,NaN,NaN,1.518812e+18,1.518812e+18,"{'contributors': None, 'coordinates': None, 'c...","{'display': 'twitter.com/ambreenpti1/st…', 'ex...",NaN,"{'display_text_range': array([ 0, 137]), 'ent...",2022-04-26
997,997,2022-04-26 10:37:47+00:00,1518902151510208514,1518902151510208512,@SSEHBAI1 محترم سچی بات کریں گے تو متوالے ساتھ...,"<a href=""http://twitter.com/download/iphone"" r...",False,1.518745e+18,1.518745e+18,234092394.0,...,[10 78],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26
998,998,2022-04-26 10:37:47+00:00,1518902151128489986,1518902151128489984,Those hands should be in handcuffs #امپورٹڈ_حک...,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,NaN,NaN,NaN,...,[ 0 83],{'media': array([{'additional_media_info': Non...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   index                      1000 non-null   int64  
 1   created_at                 1000 non-null   object 
 2   id                         1000 non-null   int64  
 3   id_str                     1000 non-null   int64  
 4   text                       1000 non-null   object 
 5   source                     1000 non-null   object 
 6   truncated                  1000 non-null   bool   
 7   in_reply_to_status_id      73 non-null     float64
 8   in_reply_to_status_id_str  73 non-null     float64
 9   in_reply_to_user_id        75 non-null     float64
 10  in_reply_to_user_id_str    75 non-null     float64
 11  in_reply_to_screen_name    75 non-null     object 
 12  user                       1000 non-null   object 
 13  geo                        0 non-null      float6

In [6]:
columns_list = list(df.columns)
print(columns_list)

['index', 'created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms', 'display_text_range', 'extended_entities', 'possibly_sensitive', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status', 'quoted_status_permalink', 'withheld_in_countries', 'extended_tweet', 'created_day']


In [7]:
# fetch few random instances of data from a specific column
random_instances = df['text'].sample(n=5)
random_instances

384    RT @ImranRaizKhan: سوشل میڈیا کا دور ہے\n\nغیر...
635    RT @Asma2936Ali: #TikTok\n#امپورٹڈ_حکومت_نامنظ...
809                   Hahahahaha\n#امپورٹڈ_حکومت_نامنظور
401    RT @ShafiqueArajput: جعلی روس اردو والوں کو یہ...
347    RT @QasimKhanSuri: Biggest trend ever analyzed...
Name: text, dtype: object

In [8]:
pd.set_option('display.width', 2000)
print(df['text'].iloc[39])

RT @mshaanshahid: Rqst frm a 🇵🇰:to finance min..kindly share details of the new negotiations done with the IMF ,the people of Pakistan are…


In [9]:
print(df['text'].iloc[49])

@EjazChaudhary @usmansaeedbasra @PTIofficial #ECP_Disqualify_Lotas #MarchAgainstImportedGovt #امپورٹڈ_حکومت_نامنظور… https://t.co/cgfM4a6jE1


In [10]:
!pip install langdetect

In [11]:
import pandas as pd
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return 'unknown'

df['language'] = df['text'].apply(detect_language)

# count the number of records for each language
language_counts = df['language'].value_counts()

# calculate the split factor
urdu_count = language_counts.get('ur', 0)
english_count = language_counts.get('en', 0)
total_count = len(df)

urdu_split_factor = urdu_count / total_count
english_split_factor = english_count / total_count
bilingual =  1.00 - (urdu_split_factor + english_split_factor)

print(f"Urdu split factor: {urdu_split_factor:.2f}")
print(f"English split factor: {english_split_factor:.2f}")
print(f"Bilingual split factor: {bilingual:.2f}")

Urdu split factor: 0.72
English split factor: 0.23
Bilingual split factor: 0.04


PREPROCESSING

In [12]:
pip install nltk

In [13]:
#function for removing not needed data
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stopwords(text):
    stopwords_set = set(stopwords.words("english"))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stopwords_set]
    return " ".join(filtered_words)

def remove_hashtags(text):
    # Define the pattern for detecting hashtags until the next whitespace
    hashtag_pattern = r'\B#\w*[a-zA-Z]+\w*'

    # Remove hashtags from the text
    cleaned_text = re.sub(hashtag_pattern, '', text)

    return cleaned_text

def remove_urdu_hashtags(text):
    # Define the pattern for detecting Urdu hashtags until the next whitespace
    urdu_hashtag_pattern = r'#[\u0600-\u06FF]+(?=\s)'

    # Remove Urdu hashtags from the text
    cleaned_text = re.sub(urdu_hashtag_pattern, '', text)

    return cleaned_text

def remove_mentions(text):
    # Define the pattern for detecting mentions until the next whitespace
    mention_pattern = r'@\w+'

    # Remove mentions from the text
    cleaned_text = re.sub(mention_pattern, '', text)

    return cleaned_text

from nltk.stem import PorterStemmer
nltk.download('punkt')

# Initialize the Porter Stemmer
stemmer = PorterStemmer()

def stem_words(text):
    # Tokenize the text into words
    words = nltk.word_tokenize(text)

    # Stem each word and collect the stems
    stemmed_words = [stemmer.stem(word) for word in words]

    # Join the stemmed words back into a single string
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def remove_text_with_numbers(text):
    # Define the pattern for detecting text containing numbers
    text_with_numbers_pattern = r'\b(?:[a-zA-Z]*\d+[a-zA-Z]*)\b'

    # Remove text containing numbers from the text
    cleaned_text = re.sub(text_with_numbers_pattern, '', text)

    return cleaned_text

def preprocess_text(text):
    # remove hashtags
    text = remove_hashtags(text)

    # remove mentions / tags
    text = remove_mentions(text)

    #remove all special characters, excluding Urdu characters
    text = re.sub(r'[^a-zA-Z0-9\s\u0600-\u06FF]', '', text)

    #remove links
    text = re.sub(r'https?://[^ ]*', '', text)
    text = re.sub(r'https?[^ ]*', '', text)
    text = re.sub(r'http[^ ]*', '', text)

    #remove "RT"
    text = re.sub(r'RT', '', text)

    # remove stopwords
    text = remove_stopwords(text)

    # remove words with numbers and numbers
    text = remove_text_with_numbers(text)

    # lemmatize / stem the words
    text = stem_words(text)

    return text.strip()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
#storing first 5 records in variable
result = df['text'].head(5).tolist()
result

['RT @fawadchaudhry: آج الیکشن کمیشن کے دفاتر کے باہر پورے ملک میں صرف ٹوکن احتجاج ہے اور بیوقوف حکومت پورا ملک ہی بند کر کے بیٹھ گئ ہے، اسلا…',
 'RT @ImranRiazKhan: عمران خان کی تقریر کا پاکستان کے 90 فیصد میڈیا میں بلیک آؤٹ',
 'RT @InsafPK: پاکستان تحریک انصاف پنجاب کا صوبائی صدر شفقت محمود اور انفارمیشن سیکرٹری مسرت جمشید چیمہ کی قیادت میں لاہور میں الیکشن کمیشن ک…',
 'RT @ZBasharit: پاکستان رمضان کے مہینے میں آزاد ہُوا تھا اور رمضان کے مہینے میں ہی ہمیں پتہ چلا کہ نہیں ابھی آزاد نہیں ہُوا \nتو مہربانی کر ک…',
 'RT @ImranRiazKhan: مبینہ طور پر ٹویٹر کی تاریخ کا سب سے بڑا ٹرینڈ #امپورٹڈ_حکومت_نامنظور بن گیا ہے۔ ہن آرام اے']

In [15]:
#checking pre-processing on first 5 records
preprocessed_result = [preprocess_text(remove_urdu_hashtags(text)) for text in result]

for item in preprocessed_result:
    print(item)

آج الیکشن کمیشن کے دفاتر کے باہر پورے ملک میں صرف ٹوکن احتجاج ہے اور بیوقوف حکومت پورا ملک ہی بند کر کے بیٹھ گئ ہے، اسلا
عمران خان کی تقریر کا پاکستان کے فیصد میڈیا میں بلیک آؤٹ
پاکستان تحریک انصاف پنجاب کا صوبائی صدر شفقت محمود اور انفارمیشن سیکرٹری مسرت جمشید چیمہ کی قیادت میں لاہور میں الیکشن کمیشن ک
پاکستان رمضان کے مہینے میں آزاد ہُوا تھا اور رمضان کے مہینے میں ہی ہمیں پتہ چلا کہ نہیں ابھی آزاد نہیں ہُوا تو مہربانی کر ک
مبینہ طور پر ٹویٹر کی تاریخ کا سب سے بڑا ٹرینڈ امپورٹڈحکومتنامنظور بن گیا ہے۔ ہن آرام اے


In [16]:
#applying the pre-process function on the entire column
df['text'] = df['text'].apply(preprocess_text)

In [17]:
random_instances = df['text'].sample(n=5)
random_instances

45     pti protest front elect commiss pakistan offic...
74     فوری بنچ فوری شنوائی شریف خاندان کے لیے انصاف ...
273    وزیرِ اعلیٰ پنجاب سردار عثمان بزدار امپورٹڈحکو...
52     fals ridicul excus import govt hide mismanag e...
85     اسکو کہتے ہیں سوشل میڈیا کی طاقت۔ ٹرینڈ پھر نم...
Name: text, dtype: object

TRANSLATION

In [18]:
pip install deep-translator

In [19]:
#testing googletranslator library
from deep_translator import GoogleTranslator

lator = GoogleTranslator()

res1 = lator.translate('یہ ایک گاڑی ہے۔#')
res2 = lator.translate('Type in English - اردو میں ٹائپ کریں')

print(res1)

This is a vehicle.#


In [20]:
#testing on only first 5 PRE-PROCESSED records
from deep_translator import GoogleTranslator

lator = GoogleTranslator()

#translated_records = [lator.translate(record) for record in result]

#for record in translated_records:
#    print(record)

#processes them one by one
def next_record(preprocessed_result):
    for row in preprocessed_result:
        yield row

translated = []
for record in next_record(preprocessed_result):
    rec = lator.translate(record)
    translated.append(rec)
    print(rec)

Today, there are only token protests in the entire country outside the offices of the Election Commission and the stupid government has shut down the entire country.
Imran Khan's speech blacked out in Pakistan's media
Pakistan Tehreek-e-Insaf Punjab Provincial President Shafqat Mahmood and Information Secretary Musrat Jamshed Cheema led the Election Commission in Lahore
Pakistan was freed in the month of Ramadan and we found out in the month of Ramadan that no, it has not been freed yet.
Arguably the biggest trend in Twitter history has become imported government disapproval. Oh relax


In [21]:
df

,index,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,withheld_in_countries,extended_tweet,created_day,language
0,0,2022-04-26 10:37:27+00:00,1518902068735533056,1518902068735533056,آج الیکشن کمیشن کے دفاتر کے باہر پورے ملک میں ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26,ur
1,1,2022-04-26 10:37:27+00:00,1518902068743913472,1518902068743913472,عمران خان کی تقریر کا پاکستان کے فیصد میڈیا می...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26,ur
2,2,2022-04-26 10:37:27+00:00,1518902068878192640,1518902068878192640,پاکستان تحریک انصاف پنجاب کا صوبائی صدر شفقت م...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26,ur
3,3,2022-04-26 10:37:27+00:00,1518902068953681922,1518902068953681920,پاکستان رمضان کے مہینے میں آزاد ہُوا تھا اور ر...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26,ur
4,4,2022-04-26 10:37:27+00:00,1518902069050380290,1518902069050380288,مبینہ طور پر ٹویٹر کی تاریخ کا سب سے بڑا ٹرینڈ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26,ur
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,2022-04-26 10:37:47+00:00,1518902151338020864,1518902151338020864,امپورٹڈحکومتنامنظور,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,{'media': array([{'additional_media_info': Non...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26,ur
996,996,2022-04-26 10:37:47+00:00,1518902151191351297,1518902151191351296,نیوٹرل بھڑوے باجوے نے مدد کی چوروں کی اور اپنی...,"<a href=""http://twitter.com/download/iphone"" r...",True,NaN,NaN,NaN,...,NaN,NaN,1.518812e+18,1.518812e+18,"{'contributors': None, 'coordinates': None, 'c...","{'display': 'twitter.com/ambreenpti1/st…', 'ex...",NaN,"{'display_text_range': array([ 0, 137]), 'ent...",2022-04-26,ur
997,997,2022-04-26 10:37:47+00:00,1518902151510208514,1518902151510208512,محترم سچی بات کریں گے تو متوالے ساتھ پائیں گے ...,"<a href=""http://twitter.com/download/iphone"" r...",False,1.518745e+18,1.518745e+18,234092394.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26,ur
998,998,2022-04-26 10:37:47+00:00,1518902151128489986,1518902151128489984,hand handcuff امپورٹڈحکومتنامنظور,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,NaN,NaN,NaN,...,{'media': array([{'additional_media_info': Non...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26,en


In [22]:
#translating the entire text column
def next_record(result):
    for row in result:
        yield row

translated = []

for record in next_record(df['text']):
    try:
        if not record.isascii():  #check if text is in English or not, so it will remain the same
            rec = remove_urdu_hashtags(record) # remove urdu hashtags if the record is not in english
            rec = lator.translate(record) #if its not english, it will be translated to english

            if not rec.isascii():
                raise Exception('Text to be translated is in an undetected language.')

        rec = preprocess_text(rec)
        translated.append(rec)
        print(rec)

    except Exception as e:
        print(f"Error translating record: {e}")
        translated.append("")  # Append an empty string or a default value

df['translated_text'] = translated

today token protest entir countri outsid offic elect commiss stupid govern shut entir countri
imran khan speech black pakistan media
pakistan tehreekeinsaf punjab provinci presid shafqat mahmood inform secretari musrat jamsh cheema led elect commiss lahor
pakistan freed month ramadan found month ramadan freed yet
arguabl biggest trend twitter histori becom import govern disapprov oh relax
import govt approv error rectif elect
kar b kaya sakta import govern disapprov
bow anyon except allah nation accept import govern chairman imran khan
import govern disapprov
imran khan done work favor neither demand
keep pictur two day later intellectu say niazi sahib went swim alon sea perform umrah
amrab almaroof import part basi islam never talk quran vers must said imran khan use quran vers polit
mani peopl come pmln ralli import prime minist take offici visit saudi arabia govern expens
whether sabir shakir imran khan anchor take money prostitut
latest well shahbaz sharif armi therefor hope earli 

In [23]:
df

,index,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,withheld_in_countries,extended_tweet,created_day,language,translated_text
0,0,2022-04-26 10:37:27+00:00,1518902068735533056,1518902068735533056,آج الیکشن کمیشن کے دفاتر کے باہر پورے ملک میں ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26,ur,today token protest entir countri outsid offic...
1,1,2022-04-26 10:37:27+00:00,1518902068743913472,1518902068743913472,عمران خان کی تقریر کا پاکستان کے فیصد میڈیا می...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26,ur,imran khan speech black pakistan media
2,2,2022-04-26 10:37:27+00:00,1518902068878192640,1518902068878192640,پاکستان تحریک انصاف پنجاب کا صوبائی صدر شفقت م...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26,ur,pakistan tehreekeinsaf punjab provinci presid ...
3,3,2022-04-26 10:37:27+00:00,1518902068953681922,1518902068953681920,پاکستان رمضان کے مہینے میں آزاد ہُوا تھا اور ر...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26,ur,pakistan freed month ramadan found month ramad...
4,4,2022-04-26 10:37:27+00:00,1518902069050380290,1518902069050380288,مبینہ طور پر ٹویٹر کی تاریخ کا سب سے بڑا ٹرینڈ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26,ur,arguabl biggest trend twitter histori becom im...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,2022-04-26 10:37:47+00:00,1518902151338020864,1518902151338020864,امپورٹڈحکومتنامنظور,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26,ur,import govern disapprov
996,996,2022-04-26 10:37:47+00:00,1518902151191351297,1518902151191351296,نیوٹرل بھڑوے باجوے نے مدد کی چوروں کی اور اپنی...,"<a href=""http://twitter.com/download/iphone"" r...",True,NaN,NaN,NaN,...,NaN,1.518812e+18,1.518812e+18,"{'contributors': None, 'coordinates': None, 'c...","{'display': 'twitter.com/ambreenpti1/st…', 'ex...",NaN,"{'display_text_range': array([ 0, 137]), 'ent...",2022-04-26,ur,neutral bharv bajwa help thiev sold mother sis...
997,997,2022-04-26 10:37:47+00:00,1518902151510208514,1518902151510208512,محترم سچی بات کریں گے تو متوالے ساتھ پائیں گے ...,"<a href=""http://twitter.com/download/iphone"" r...",False,1.518745e+18,1.518745e+18,234092394.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26,ur,respect speak truth support import govern
998,998,2022-04-26 10:37:47+00:00,1518902151128489986,1518902151128489984,hand handcuff امپورٹڈحکومتنامنظور,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26,en,hand handcuff import govern disapprov


In [24]:
df = df.drop(columns=[col for col in df.columns if col != 'translated_text'])

In [25]:
df

,translated_text
0,today token protest entir countri outsid offic...
1,imran khan speech black pakistan media
2,pakistan tehreekeinsaf punjab provinci presid ...
3,pakistan freed month ramadan found month ramad...
4,arguabl biggest trend twitter histori becom im...
...,...
995,import govern disapprov
996,neutral bharv bajwa help thiev sold mother sis...
997,respect speak truth support import govern
998,hand handcuff import govern disapprov
